# Neural Network

A neural network is created which can be used for training and testing on trips data.



In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import pickle

from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

In [2]:
path_output=os.path.join(os.getcwd(), "..", "data", "output")

In [3]:
features = pd.read_csv(os.path.join(path_output, "Features.csv"))
features = features.drop("Unnamed: 0", axis=1)

### Take Params from csv

In [4]:
params = pd.read_csv(os.path.join(os.getcwd(), "..", "data", "input", "params.csv")).drop("0", axis=1)
_test_size = params[params["param"]=="test_size"]["value"].values[0]
_random_state = int(params[params["param"]=="random_state"]["value"].values[0])
_epochs = int(params[params["param"]=="epochs"]["value"].values[0])
_validation_size = params[params["param"]=="validation_size"]["value"].values[0]

### Neural Network

In [14]:
def train_neural_network(X_train_scaled, y_train, on, hex_size):
    """
    Train Neural Network Model

    Train and save a Neural Network model.
    The network has the following properties:
        - three hidden layer
        - 50 epochs
        - activation function is relu
        - dimension of input and hidden layer is 36
        - dimension of output layer is 1
        - dropout is not used
    Then evaluate the error metrics by another method.

    Args:
        X_train_scaled (DataFrame):   Scaled X input of train set (matrix)
        y_train (Series):             y output to train on (vector)
    Returns:
        nn_regression_sets (array): true y values and predicted y values for train and validation set
    """
    # create a validation set which is 20% of the whole dataset. Therefore use formula to receive ca. 0.2857.
    X_train, X_val, y_train, y_val = train_test_split(X_train_scaled, y_train, random_state=_random_state, test_size=_validation_size)
    neural_network = keras.Sequential(
        [layers.Dense(36, activation="relu", input_shape=[X_train_scaled.shape[1]], kernel_initializer="random_normal"),
         # layers.Dropout(0.2),
         layers.Dense(36, activation="relu", kernel_initializer="random_normal"),
         layers.Dense(36, activation="relu", kernel_initializer="random_normal"),
         layers.Dense(36, activation="relu", kernel_initializer="random_normal"),
         layers.Dense(36, activation="relu", kernel_initializer="random_normal"),
         layers.Dense(36, activation="relu", kernel_initializer="random_normal"),
         # layers.Dense(36, activation="relu", kernel_initializer="random_normal"),
         # layers.Dense(36, activation="relu", kernel_initializer="random_normal"),
         # layers.Dense(36, activation="softmax"),
         # layers.Dense(36, activation="softmax"),
         # layers.Dropout(0.2),
         layers.Dense(1)])
    optimizer = keras.optimizers.RMSprop(learning_rate=0.001, rho=0.9)
    neural_network.compile(loss="mse",
                           optimizer=optimizer,
                           metrics=["mae", "mse"])
    epochs = _epochs
    # create a validation set which is 20% of the whole dataset. Therefore use formula to receive ca. 0.2857.
    history = neural_network.fit(X_train, y_train, epochs=epochs, validation_data=(X_val, y_val))
    neural_network.save(os.path.join(path_output, "models", "NN_Regression_Model_"+on+"_"+hex_size))
    y_prediction_train = neural_network.predict(X_train)
    y_prediction_val = neural_network.predict(X_val)
    plot_train_loss(history, on=on, hex_size=hex_size)
    nn_regression_sets = [y_train, y_val, y_prediction_train, y_prediction_val]
    return nn_regression_sets

### Loss visualization by epoch

In [21]:
def plot_train_loss(history, on, hex_size):
    """
    Plot the train and validation loss of Neural Network.

    Args:
        history (Object): History of loss during training of neural network
        on (str): time resolution to train on
    Returns:
        No return
    """
    print("Plot training and visualization loss...")
    # Plotting the training and validation loss
    loss = history.history["loss"]
    val_loss = history.history["val_loss"]

    epochs = range(1, len(loss) + 1)
    fig, ax = plt.subplots(figsize=(16, 8), dpi=300)
    ax.plot(epochs, loss, "bo", label="Training loss")
    ax.plot(epochs, val_loss, "b", label="Validation loss")
    ax.set_title("Training and validation loss "+on+"_"+hex_size, fontsize=18)
    ax.set_xlabel("Epochs", fontsize=16)
    ax.set_ylabel("Loss", fontsize=16)
    plt.legend()
    #plt.show()
    fig.savefig(os.path.join(path_output, "NN_error_per_epoch_"+on+"_"+hex_size+".png"))
    plt.close(fig)

### Run NN

In [19]:
def train_NN(on="24_sum", hex_size="hexa_small"):
    """
    Split the data in train and test set by 0.3 test set. 
    Then Scale the data and do a PCA. 
    Last train the NN on the chosen time resolution
    
    Args:
        on (str): time resolution to train on
        
    Returns:
        No return
    """
    print("Time Resolution is", on)
    #print("Split Data with random state", _random_state, "and test size", str(_test_size)+"...")
    features_X = features.drop(["24_sum", "6_sum", "2_sum", "1_sum"], axis=1)
    features_y = features[on]
    
    # Spatial Resolution
    print("Spatial Resolution is", hex_size)
    if hex_size=="hexa_small":
        features_X = features_X.drop("hexa_big", axis=1)
    else:
        features_X = features_X.drop("hexa_small", axis=1)
    
    #Split
    X_train, X_test, y_train, y_test = train_test_split(features_X, features_y, random_state=_random_state, test_size=_test_size)

    #print("Scale", hex_size, "Data with Standard Scaler...")
    with open(os.path.join(path_output, "models", "Standard_Scaler_"+hex_size+".pkl"), "rb") as f:
        standard_scaler = pickle.load(f)
    X_train_scaled = standard_scaler.transform(X_train)

    #print("Do PCA on", hex_size, "Data...")
    with open(os.path.join(path_output, "models", "PCA_"+hex_size+".pkl"), "rb") as f:
        pca = pickle.load(f)
    X_train_transformed = pca.transform(X_train_scaled)

    print("Train", "NN_Regression_Model_"+on+"_"+hex_size, "...")
    nn_regression_sets = train_neural_network(X_train_transformed, y_train.to_numpy(), on=on, hex_size=hex_size)

In [22]:
#Train the NN for each time resolution.
hex_size = ["hexa_small", "hexa_big"]
time_resolutions = ["24_sum", "6_sum", "2_sum", "1_sum"]
for time in time_resolutions:
    for size in hex_size:
        train_NN(on=time, hex_size=size)
        print()
print("Done")
#train_NN(on="1_sum", hex_size="hexa_small")

Time Resolution is 24_sum
Split Data with random state 42 and test size 0.3...
Spatial Resolution is hexa_small
Scale hexa_small Data with Standard Scaler...
Do PCA on hexa_small Data...
Train NN 24_sum...
Train on 283719 samples, validate on 113488 samples
283719/283719 [==============================] - 74s 262us/sample - loss: 1034238.5918 - mae: 627.2525 - mse: 1034237.2500 - val_loss: 506023.5723 - val_mae: 444.5254 - val_mse: 506024.4062
INFO:tensorflow:Assets written to: C:\Users\lenovo\Documents\GitHub\AAA_Jupyter_Explorers\notebooks\..\data\output\models\NN_Regression_Model_24_sum_hexa_small\assets
Plot training and visualization loss...
Time Resolution is 24_sum
Split Data with random state 42 and test size 0.3...
Spatial Resolution is hexa_big
Scale hexa_big Data with Standard Scaler...
Do PCA on hexa_big Data...
Train NN 24_sum...
Train on 283719 samples, validate on 113488 samples
283719/283719 [==============================] - 62s 217us/sample - loss: 1282425.4757 - mae: